In [2]:
import pandas as pd
import os
import datetime
import seaborn as sns
import numpy as np

# Pre-Process Taxi Data

In [2]:
# function to pre-process taxi data
def preprocess_data(filename, arr,PATH):
    final_data = []
    
    for doc in arr:
        
        taxi = pd.read_csv(PATH + filename + doc)
        # remove impossible data such as tip less than $0. All exaplined in report
        taxi_impossible_vals_removed = taxi[(taxi['tip_amount']>=0) &\
             (taxi['trip_distance']>0) & (taxi['trip_distance']<25) &\
             (taxi['fare_amount']>=2.5) & (taxi['passenger_count']>0) &\
             (taxi['payment_type']==1)& (taxi['extra']>=0)].copy()

        # keep only the features that matter - also explained in report
        taxi_clean = taxi_impossible_vals_removed[['tpep_pickup_datetime','trip_distance','PULocationID',\
                           'DOLocationID','fare_amount','tip_amount','total_amount','extra','mta_tax',\
                           'tolls_amount','improvement_surcharge']]
        # column names of taxi data
        columns = list(taxi_clean.columns)
        
        # summary statistics of taxi data
        info = taxi_clean.describe().transpose()
        
        # remove outliers
        for index, col in info.iterrows():
            
            if index in ['VendorID', 'tpep_pickup_datetime','tpep_dropoff_datetime', 'PULocationID','DOLocationID']:
                continue
            # outliers defined as outsisde of x1.5 the Inter Quartile Range (IQR)
            IQR = col["75%"] - col["25%"]

            upper_lim = col["75%"] + (IQR * 1.5)
            lower_lim = col["25%"] - (IQR * 1.5)
            
            taxi_clean=taxi_clean.loc[taxi_clean[index]<= upper_lim]
            taxi_clean=taxi_clean.loc[taxi_clean[index]>= lower_lim]
        
        
        final_data.append(taxi_clean)
    
    final_taxi = pd.concat(final_data) 
    # return a cleaned dataframe
    return final_taxi


In [7]:
# function used to extract features of pickup date, pickup hour and pickup month from
# datetime objects. 
def get_datetime(data):
    
    data_dates = data.copy()
    # use the pickup datetime data
    data_dates['tpep_pickup_datetime'] = pd.to_datetime(data.tpep_pickup_datetime)
    # extract date
    data_dates['pickup_date'] = data_dates['tpep_pickup_datetime'].dt.date # Extract date
    # extract hour
    data_dates['pickup_hour'] = data_dates['tpep_pickup_datetime'].dt.hour
    # extract month
    data_dates['pickup_month']=data_dates['tpep_pickup_datetime'].dt.month
    # sort by date of pick up
    data_dates.sort_values(by = ['pickup_date'],inplace=True)
    # remove datetime object
    data_dates.drop(['tpep_pickup_datetime'], axis = 1, inplace = True)
    # return dataframe
    return data_dates.reset_index()

# Weather Data

In [3]:
# Function to web scrape weather data. 

# webpages saved as files here.
WEATHER_PATH = '/Users/Work/Documents/2020/Sem 2/Applied_Data_Science/Assignment2/Weather/'

import numpy as np
from bs4 import BeautifulSoup

def get_month_weather(year_month):
    
    with open(WEATHER_PATH+year_month+".html") as file:
        soup = BeautifulSoup(file.read(), "html.parser")
    tables = soup.find_all("table", "days ng-star-inserted")
    
    # Get data
    table_data = [
        [col.get_text() for col in row.find_all('td') if len(col.get_text())>10] 
        for row in tables[0].find_all("tr")
    ]
    table_data = [data for data in table_data if len(data)>0]
    table_data[1] = [col.split("  ") for col in table_data[1]]
    
    table_data.append([np.asarray(col).reshape((-1, 3)) for col in table_data[1][1:-1]])
    
    temp_table = [np.asarray(table_data[1][0])]
    
    for col in table_data[2]:
        temp_table.append(col)
        
    temp_table.append(np.asarray(table_data[1][-1]))
    
    final_table = [
        [temp_table[col][row] for col in range(len(temp_table))] 
        for row in range(len(temp_table[1]))
    ]
    
    final_table = [
        [final_table[row][0]] +
        [item for sublist in final_table[row][1:-1] for item in sublist] +
        [final_table[row][-1]]
        for row in range(len(final_table))
    ]
    
    final_table.insert(0, ['Day'] +
                       [col_name for duplicate_cols in [
                           [item, item, item] for item in table_data[0][:-1]
                       ] for col_name in duplicate_cols] +
                       [table_data[0][-1]])
    
    return final_table

In [4]:
def weather_per_year(year):
    
    df_lst = []
    for i in range(1,13):
        # access each months weather data. 
        month = str(year + i)
        
        # call get month weather data to convert to a table
        tmp = get_month_weather(month)
        
        if i == 1:
            row = 1       
        else:
            row = 2
        # convert to dataframe   
        df = pd.DataFrame(tmp[row:-1], columns=tmp[0])
            
        # create month column
        df["Month"] = i
        # append to list of dataframes for each month
        df_lst.append(df)
        
    # concatenate all the months to form a dataframe of the years worth of weather data
    weather = pd.concat(df_lst).reset_index()
    
    # return weather data for the year
    return weather


In [2]:

def create_date_time(year,weather_data):
    
    # create new column called date in weather data
    weather_data['Date'] = ""
    
    for index,col in weather_data.iterrows():
    
        if index == 0:
            continue
        
        # extract date and date 
        weather_data.loc[index, 'Date'] = datetime.date(year,col['Month'],int(col['Day']))
    # keep only tempreature, windspeed and precipitation 
    weather_data = weather_data[['Temperature (° F)','Wind Speed (mph)','Precipitation (in)','Date']]
    
    # return weather data
    return weather_data

# Final Pre-Processing 

In [ ]:
def final_preprocess(PATH,filename):
    
    data = pd.read_csv(PATH + filename)
    
    # remove extra columns 
    data.drop(['Unnamed: 0'], axis = 1, inplace = True)
    data.drop(['index'], axis = 1, inplace = True)
    
    return data

# Create response variable 

In [ ]:
def create_target(data):
    # group by location of pickup, the date and then the hour
    data_by_hour = data.groupby(['PULocationID','pickup_date','pickup_hour']).mean().reset_index()
    # find how many trips occured in each pick up zone, for each hour on each day 
    trips_per_hour = data.groupby(['PULocationID','pickup_date','pickup_hour']).size().reset_index()
    # trips per hour is the response variable 
    data_by_hour['Trips_per_hour'] = trips_per_hour[0]
    
    return data_by_hour